#### Imports 

In [28]:
from time import gmtime, strftime
from sagemaker import image_uris
from utils import InferenceSpec
import sagemaker
import logging
import boto3
import json

#### Setup

In [29]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [30]:
region = sagemaker.Session().boto_region_name
boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client('sagemaker')
logger.info(f'Region = {region}')

Region = us-east-1
Region = us-east-1


### List all Model Package Groups in SageMaker Model Registry

In [31]:
sagemaker_client.list_model_package_groups()['ModelPackageGroupSummaryList']

[{'ModelPackageGroupName': 'bert-email-classifier',
  'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package-group/bert-email-classifier',
  'ModelPackageGroupDescription': 'BERT-based multi-class classifier for email threat detection',
  'CreationTime': datetime.datetime(2021, 8, 8, 4, 12, 23, 602000, tzinfo=tzlocal()),
  'ModelPackageGroupStatus': 'Completed'},
 {'ModelPackageGroupName': 'email-spam-classification',
  'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package-group/email-spam-classification',
  'ModelPackageGroupDescription': 'email spam',
  'CreationTime': datetime.datetime(2021, 8, 5, 17, 1, 6, 613000, tzinfo=tzlocal()),
  'ModelPackageGroupStatus': 'Completed'},
 {'ModelPackageGroupName': 'project-1-ci-cd-p-wphdcrsekdft',
  'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package-group/project-1-ci-cd-p-wphdcrsekdft',
  'CreationTime': datetime.datetime(2021, 7, 26, 13, 20, 29, 726000, tzinfo=tzl

### Create a Model Package Group

In [32]:
model_package_group_name = 'bert-email-classifier'
config = {'ModelPackageGroupName': model_package_group_name, 
          'ModelPackageGroupDescription': 'BERT-based multi-class classifier for email threat detection'
         }

In [33]:
sagemaker_client.list_model_package_groups(NameContains=model_package_group_name)['ModelPackageGroupSummaryList']

[{'ModelPackageGroupName': 'bert-email-classifier',
  'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package-group/bert-email-classifier',
  'ModelPackageGroupDescription': 'BERT-based multi-class classifier for email threat detection',
  'CreationTime': datetime.datetime(2021, 8, 8, 4, 12, 23, 602000, tzinfo=tzlocal()),
  'ModelPackageGroupStatus': 'Completed'}]

In [34]:
if not sagemaker_client.list_model_package_groups(NameContains=model_package_group_name)['ModelPackageGroupSummaryList']:
    response = sagemaker_client.create_model_package_group(**config)
    logger.info(f'Model Package Group: {model_package_group_name} created!')
    logger.info(f'Response: {response}')
else:
    logger.warning('Model Package Group already exists!')

Model Package Group already exists!
Model Package Group already exists!


### List Model Packages 
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.list_model_packages

In [35]:
response = sagemaker_client.list_model_packages(ModelPackageGroupName=model_package_group_name)
response

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'bert-email-classifier',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/1',
   'ModelPackageDescription': 'CreatedOn: 08-08-04-12',
   'CreationTime': datetime.datetime(2021, 8, 8, 4, 13, 20, 734000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'Approved'}],
 'ResponseMetadata': {'RequestId': '8bfe52c2-c31a-446a-872d-0c72c493a1ff',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8bfe52c2-c31a-446a-872d-0c72c493a1ff',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '351',
   'date': 'Sun, 08 Aug 2021 04:14:16 GMT'},
  'RetryAttempts': 0}}

### Describe a Model Package Group

In [36]:
sagemaker_client.describe_model_package_group(ModelPackageGroupName=model_package_group_name)

{'ModelPackageGroupName': 'bert-email-classifier',
 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package-group/bert-email-classifier',
 'ModelPackageGroupDescription': 'BERT-based multi-class classifier for email threat detection',
 'CreationTime': datetime.datetime(2021, 8, 8, 4, 12, 23, 602000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:892313895307:user-profile/d-dowart1jabkf/ts-zd-e2e',
  'UserProfileName': 'ts-zd-e2e',
  'DomainId': 'd-dowart1jabkf'},
 'ModelPackageGroupStatus': 'Completed',
 'ResponseMetadata': {'RequestId': '7670b2a2-d52f-47e4-997e-919a51a12269',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7670b2a2-d52f-47e4-997e-919a51a12269',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '491',
   'date': 'Sun, 08 Aug 2021 04:14:16 GMT'},
  'RetryAttempts': 0}}

### How do I register a trained BERT model to the model registry 

In [37]:
ecr_image = image_uris.retrieve(framework='huggingface', 
                                region='us-east-1', 
                                version='4.6.1', 
                                image_scope='inference', 
                                base_framework_version='tensorflow2.4.1', 
                                py_version='py37', 
                                container_version='ubuntu18.04', 
                                instance_type='ml.m5.4xlarge')
ecr_image

'763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4.1-transformers4.6.1-cpu-py37-ubuntu18.04'

1. Define the inference spec 

In [38]:
# ecr_image = '763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-tensorflow-training:2.4.1-transformers4.6.1-gpu-py37-cu110-ubuntu18.04'
inference_spec = InferenceSpec().get_dict(ecr_image=ecr_image, 
                                          supports_gpu=True, 
                                          supported_content_types=['text/csv'], 
                                          supported_mime_types=['text/csv'])

In [39]:
inference_spec["InferenceSpecification"]["Containers"][0]["ModelDataUrl"] = 's3://sagemaker-us-east-1-892313895307/pipeline/model/model.tar.gz'
inference_spec

{'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4.1-transformers4.6.1-cpu-py37-ubuntu18.04',
    'ModelDataUrl': 's3://sagemaker-us-east-1-892313895307/pipeline/model/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xlarge',
   'ml.m5.2xlarge',
   'ml.m5.4xlarge',
   'ml.m5.12xlarge',
   'ml.m5.24xlarge',
   'ml.c4.xlarge',
   'ml.c4.2xlarge',
   'ml.c4.4xlarge',
   'ml.c4.8xlarge',
   'ml.c5.xlarge',
   'ml.c5.2xlarge',
   'ml.c5.4xlarge',
   'ml.c5.9xlarge',
   'ml.c5.18xlarge',
   'ml.p2.xlarge',
   'ml.p2.8xlarge',
   'ml.p2.16xlarge',
   'ml.p3.2xlarge',
   'ml.p3.8xlarge',
   'ml.p3.16xlarge'],
  'SupportedRealtimeInferenceInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xl

**Note:** The ModelDataUrl can also be obtained from a previously run training job.

2. Define/load model metrics 

In [40]:
with open('./metrics.json', 'rb') as f:
    model_metrics = json.load(f)

In [41]:
model_metrics

{'binary_classification_metrics': {'accuracy': {'value': 0.7768782067298889,
   'standard_deviation': 'NaN'},
  'f2': {'value': 0.5370374250411987, 'standard_deviation': 'NaN'}}}

In [42]:
current_timestamp = strftime('%m-%d-%H-%M', gmtime())

Let's push this metrics file to S3

In [43]:
!aws s3 cp metrics.json s3://sagemaker-us-east-1-892313895307/metrics/{current_timestamp}/metrics.json

upload: ./metrics.json to s3://sagemaker-us-east-1-892313895307/metrics/08-08-04-14/metrics.json


In [44]:
model_metrics = {
    "ModelQuality": {
        "Statistics": {
            "ContentType": "application/json",
            "S3Uri": f"s3://sagemaker-us-east-1-892313895307/metrics/{current_timestamp}/metrics.json",
        }
    }
}

3. Create model package config (dictionary)

In [45]:
model_package_input_dict = {"ModelPackageGroupName": model_package_group_name,
                            "ModelPackageDescription": f"CreatedOn: {current_timestamp}", 
                            "ModelApprovalStatus": "PendingManualApproval", 
                            "ModelMetrics": model_metrics}

model_package_input_dict.update(inference_spec)
model_package_input_dict

{'ModelPackageGroupName': 'bert-email-classifier',
 'ModelPackageDescription': 'CreatedOn: 08-08-04-14',
 'ModelApprovalStatus': 'PendingManualApproval',
 'ModelMetrics': {'ModelQuality': {'Statistics': {'ContentType': 'application/json',
    'S3Uri': 's3://sagemaker-us-east-1-892313895307/metrics/08-08-04-14/metrics.json'}}},
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4.1-transformers4.6.1-cpu-py37-ubuntu18.04',
    'ModelDataUrl': 's3://sagemaker-us-east-1-892313895307/pipeline/model/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xlarge',
   'ml.m5.2xlarge',
   'ml.m5.4xlarge',
   'ml.m5.12xlarge',
   'ml.m5.24xlarge',
   'ml.c4.xlarge',
   'ml.c4.2xlarge',
   'ml.c4.4xlarge',
   'ml.c4.8xlarge',
   'ml.c5.xlarge',
   'ml.c5.2xlarge',
   'ml.c5.4xlarge',
   'ml.c

In [46]:
meta = {'MetadataProperties': {'GeneratedBy': 'BERT-Pipeline'}}
model_package_input_dict.update(meta)

In [47]:
model_package_input_dict

{'ModelPackageGroupName': 'bert-email-classifier',
 'ModelPackageDescription': 'CreatedOn: 08-08-04-14',
 'ModelApprovalStatus': 'PendingManualApproval',
 'ModelMetrics': {'ModelQuality': {'Statistics': {'ContentType': 'application/json',
    'S3Uri': 's3://sagemaker-us-east-1-892313895307/metrics/08-08-04-14/metrics.json'}}},
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4.1-transformers4.6.1-cpu-py37-ubuntu18.04',
    'ModelDataUrl': 's3://sagemaker-us-east-1-892313895307/pipeline/model/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xlarge',
   'ml.m5.2xlarge',
   'ml.m5.4xlarge',
   'ml.m5.12xlarge',
   'ml.m5.24xlarge',
   'ml.c4.xlarge',
   'ml.c4.2xlarge',
   'ml.c4.4xlarge',
   'ml.c4.8xlarge',
   'ml.c5.xlarge',
   'ml.c5.2xlarge',
   'ml.c5.4xlarge',
   'ml.c

4. Create the Model Package 

In [48]:
response = sagemaker_client.create_model_package(**model_package_input_dict)
response

{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/2',
 'ResponseMetadata': {'RequestId': 'da23445a-130b-4c81-b1a7-2f21589df78b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'da23445a-130b-4c81-b1a7-2f21589df78b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Sun, 08 Aug 2021 04:14:21 GMT'},
  'RetryAttempts': 0}}

In [49]:
arn = response["ModelPackageArn"]
model_package_info = sagemaker_client.describe_model_package(ModelPackageName=arn)
model_package_info

{'ModelPackageGroupName': 'bert-email-classifier',
 'ModelPackageVersion': 2,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/2',
 'ModelPackageDescription': 'CreatedOn: 08-08-04-14',
 'CreationTime': datetime.datetime(2021, 8, 8, 4, 14, 22, 252000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4.1-transformers4.6.1-cpu-py37-ubuntu18.04',
    'ImageDigest': 'sha256:8bf391a8d10e8e7f5d26223625e8ab97276fd7dfa2a41075ddd84418055ecbb8',
    'ModelDataUrl': 's3://sagemaker-us-east-1-892313895307/pipeline/model/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xlarge',
   'ml.m5.2xlarge',
   'ml.m5.4xlarge',
   'ml.m5.12xlarge',
   'ml.m5.24xlarge',
   'ml.c4.xlarge',
   'ml.c4.2xlarge',
   'ml.c4.4xlarge',
   

In [50]:
status = model_package_info["ModelPackageStatus"]

while status not in ["Completed", "Failed"]:
    time.sleep(5)
    info = sagemaker_client.describe_model_package(ModelPackageName=arn)
    status = info["ModelPackageStatus"]
    logger.info(f"model package status: {status}")
logger.info(f"model package status: {status}")

model package status: Completed
model package status: Completed


### Describe a Model Package 

In [51]:
arn = model_package_info['ModelPackageArn']
sagemaker_client.describe_model_package(ModelPackageName=arn)

{'ModelPackageGroupName': 'bert-email-classifier',
 'ModelPackageVersion': 2,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/2',
 'ModelPackageDescription': 'CreatedOn: 08-08-04-14',
 'CreationTime': datetime.datetime(2021, 8, 8, 4, 14, 22, 252000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-tensorflow-inference:2.4.1-transformers4.6.1-cpu-py37-ubuntu18.04',
    'ImageDigest': 'sha256:8bf391a8d10e8e7f5d26223625e8ab97276fd7dfa2a41075ddd84418055ecbb8',
    'ModelDataUrl': 's3://sagemaker-us-east-1-892313895307/pipeline/model/model.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m4.xlarge',
   'ml.m4.2xlarge',
   'ml.m4.4xlarge',
   'ml.m4.10xlarge',
   'ml.m4.16xlarge',
   'ml.m5.large',
   'ml.m5.xlarge',
   'ml.m5.2xlarge',
   'ml.m5.4xlarge',
   'ml.m5.12xlarge',
   'ml.m5.24xlarge',
   'ml.c4.xlarge',
   'ml.c4.2xlarge',
   'ml.c4.4xlarge',
   

### Update status of a Model Package 

In [52]:
model_package_arn = 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/1'

In [53]:
model_package_update_input_dict = {"ModelPackageArn" : model_package_arn, 
                                   "ModelApprovalStatus" : "Approved"}
model_package_update_response = sagemaker_client.update_model_package(**model_package_update_input_dict)
model_package_update_response

{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:892313895307:model-package/bert-email-classifier/1',
 'ResponseMetadata': {'RequestId': '3ebd6cd3-f0ad-456d-877a-ffce545b67ef',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3ebd6cd3-f0ad-456d-877a-ffce545b67ef',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Sun, 08 Aug 2021 04:14:28 GMT'},
  'RetryAttempts': 0}}

### Delete all Model Packages in a Model Package Group

List all standalone model packages (ones that are not affiliated to any Model Package Group).

In [ ]:
response = sagemaker_client.list_model_packages()
response

Delete all standalone model packages 

In [ ]:
for model_package in response['ModelPackageSummaryList']:
    logger.info(f'Model Pacakge: {model_package}')
    # package_name = model_package['ModelPackageName']
    package_arn = model_package['ModelPackageArn']
    response = sagemaker_client.delete_model_package(ModelPackageName=package_arn)
    logger.info(f'Response: {response}')

List model packages that are affiliated to our Model Package Group

In [ ]:
response = sagemaker_client.list_model_packages(ModelPackageGroupName=model_package_group_name)
response

In [ ]:
for model_package in response['ModelPackageSummaryList']:
    logger.info(f'Model Package: {model_package}')
    # package_name = model_package['ModelPackageName']
    package_arn = model_package['ModelPackageArn']
    response = sagemaker_client.delete_model_package(ModelPackageName=package_arn)
    logger.info(f'Response: {response}')

### Delete a Model Package Group if needed

**Note:** We need to delete all Model Packages inside the Model Package Group first to delete the package group.

In [ ]:
sagemaker_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)

### Recreate the BERT model from model registry and use it to create a real-time inference endpoint

### How do I create an endpoint using model artifacts in S3 